In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from math import pi
from datetime import datetime  as dt
import matplotlib
import networkx as nx

# Import API key
from api_keys import api_key

In [3]:
# Dependencies
# ----------------------------------
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine

# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base

# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 

In [4]:
# Create City/location and transport Classes
# ----------------------------------

# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()


# Creates Classes which will serve as the anchor points for our Tables
class State(Base):
    __tablename__ = 'location'
    id = Column(Integer, primary_key=True)
    city_name = Column(String(255))
    date = Column(String(255))
    cases = Column(Integer)
    deaths = Column(Integer)
    new_cases = Column(Integer)
    new_deaths = Column(Integer)

In [5]:
covid_cases = "all_state_csv.csv"
covid_mobility = "google_mobility/us-mobility.csv"
covid_regional_mobility = "google_mobility/regional-mobility.csv"

In [7]:
cases_df = pd.read_csv(covid_cases)
mobility_df = pd.read_csv(covid_regional_mobility)

In [8]:
mobility_df

,country,region,date,retail,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
0,United Arab Emirates,Total,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,United Arab Emirates,Total,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,United Arab Emirates,Total,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,United Arab Emirates,Total,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,United Arab Emirates,Total,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...
78727,Zimbabwe,Total,2020-04-13,-71.0,-63.0,-51.0,-83.0,-80.0,49.0
78728,Zimbabwe,Total,2020-04-14,-63.0,-53.0,-47.0,-78.0,-64.0,41.0
78729,Zimbabwe,Total,2020-04-15,-62.0,-50.0,-51.0,-77.0,-65.0,42.0
78730,Zimbabwe,Total,2020-04-16,-62.0,-50.0,-48.0,-77.0,-64.0,42.0


In [25]:
mobility_US = mobility_df.loc[mobility_df['country'] == "United States"]

In [27]:
mobility_US.rename(columns = {'region':'state'}, inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [28]:
mobility_oregon = mobility_US.loc[mobility_US['state'] == "Oregon"]

In [29]:
mobility_oregon

,country,state,date,retail,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
75729,United States,Oregon,2020-02-15,2.0,-3.0,-5.0,-1.0,-1.0,0.0
75730,United States,Oregon,2020-02-16,14.0,3.0,33.0,3.0,0.0,-1.0
75731,United States,Oregon,2020-02-17,10.0,2.0,74.0,-3.0,-25.0,4.0
75732,United States,Oregon,2020-02-18,4.0,2.0,42.0,5.0,1.0,0.0
75733,United States,Oregon,2020-02-19,5.0,3.0,51.0,6.0,1.0,-1.0
75734,United States,Oregon,2020-02-20,4.0,4.0,33.0,4.0,1.0,-1.0
75735,United States,Oregon,2020-02-21,4.0,1.0,37.0,7.0,4.0,-1.0
75736,United States,Oregon,2020-02-22,6.0,3.0,72.0,9.0,3.0,-2.0
75737,United States,Oregon,2020-02-23,10.0,2.0,-11.0,3.0,1.0,0.0
75738,United States,Oregon,2020-02-24,5.0,1.0,28.0,3.0,4.0,-1.0


In [30]:
state_list = cases_df['state'].unique()


In [38]:
#for state in state_list:
df1 = cases_df.loc[cases_df['state'] == 'Alabama']
df2 = mobility_US.loc[mobility_US['state'] == 'Alabama']
#df2.rename(columns = {'region':'state'}, inplace = True)
#df3 = df1.join(df2,on='date',how='inner')
df3 = pd.merge(df1, df2, on='date')

In [39]:
df3

,index_row,date,state_x,fips,cases,deaths,new_cases,new_deaths,country,state_y,retail,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
0,1667,2020-03-13,Alabama,1,6,0,6,0,United States,Alabama,7.0,32.0,26.0,7.0,-2.0,0.0
1,1668,2020-03-14,Alabama,1,12,0,6,0,United States,Alabama,1.0,28.0,55.0,12.0,4.0,0.0
2,1669,2020-03-15,Alabama,1,23,0,11,0,United States,Alabama,-7.0,16.0,16.0,6.0,-4.0,2.0
3,1670,2020-03-16,Alabama,1,29,0,6,0,United States,Alabama,-2.0,24.0,22.0,2.0,-10.0,4.0
4,1671,2020-03-17,Alabama,1,39,0,10,0,United States,Alabama,-11.0,17.0,25.0,-1.0,-17.0,7.0
5,1672,2020-03-18,Alabama,1,51,0,12,0,United States,Alabama,-13.0,13.0,39.0,-3.0,-22.0,8.0
6,1673,2020-03-19,Alabama,1,78,0,27,0,United States,Alabama,-20.0,15.0,27.0,-8.0,-25.0,10.0
7,1674,2020-03-20,Alabama,1,106,0,28,0,United States,Alabama,-27.0,10.0,7.0,-14.0,-29.0,14.0
8,1675,2020-03-21,Alabama,1,131,0,25,0,United States,Alabama,-35.0,5.0,10.0,-13.0,-16.0,8.0
9,1676,2020-03-22,Alabama,1,157,0,26,0,United States,Alabama,-40.0,-7.0,2.0,-22.0,-27.0,9.0


In [35]:
df_all = pd.merge(cases_df, mobility_US, on=['date','state']).fillna(0)

In [36]:
df_all

,index_row,date,state,fips,cases,deaths,new_cases,new_deaths,country,retail,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
0,25,2020-02-15,Washington,53,1,0,0,0,United States,2.0,-1.0,8.0,8.0,-3.0,0.0
1,26,2020-02-16,Washington,53,1,0,0,0,United States,15.0,3.0,19.0,5.0,-1.0,-1.0
2,27,2020-02-17,Washington,53,1,0,0,0,United States,12.0,2.0,63.0,-7.0,-35.0,6.0
3,28,2020-02-18,Washington,53,1,0,0,0,United States,5.0,2.0,52.0,7.0,-4.0,0.0
4,29,2020-02-19,Washington,53,1,0,0,0,United States,4.0,3.0,60.0,7.0,-2.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1635,1816,2020-03-31,West Virginia,54,162,1,17,0,United States,-37.0,-15.0,-14.0,-28.0,-42.0,15.0
1636,1817,2020-04-01,West Virginia,54,191,2,29,1,United States,-34.0,-10.0,-14.0,-26.0,-43.0,15.0
1637,1818,2020-04-02,West Virginia,54,217,2,26,0,United States,-35.0,-10.0,-2.0,-31.0,-42.0,16.0
1638,1819,2020-04-03,West Virginia,54,242,2,25,0,United States,-35.0,-10.0,8.0,-36.0,-42.0,17.0


In [42]:
df_all.to_csv("transport_cases_table.csv")

In [12]:
df_all

,index_row,date,state,fips,cases,deaths,new_cases,new_deaths,region,retail_recreation,grocery_pharmacy,parks,transit_stations,workplaces,residential
0,68,2020-03-29,Washington,53,4896,207,585,16,Total,-48.0,-26.0,-11.0,-56.0,-41.0,13.0
1,68,2020-03-29,Washington,53,4896,207,585,16,Adams County,-31.0,4.0,NaN,-31.0,-27.0,NaN
2,68,2020-03-29,Washington,53,4896,207,585,16,Asotin County,-33.0,NaN,NaN,NaN,-31.0,7.0
3,68,2020-03-29,Washington,53,4896,207,585,16,Benton County,-47.0,-23.0,3.0,-12.0,-39.0,10.0
4,68,2020-03-29,Washington,53,4896,207,585,16,Chelan County,-43.0,-20.0,-15.0,-27.0,-43.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2855,1814,2020-03-29,West Virginia,54,124,1,11,1,Wayne County,-1.0,-10.0,NaN,NaN,-28.0,0.0
2856,1814,2020-03-29,West Virginia,54,124,1,11,1,Webster County,NaN,NaN,NaN,NaN,-38.0,NaN
2857,1814,2020-03-29,West Virginia,54,124,1,11,1,Wetzel County,-39.0,-15.0,NaN,NaN,-31.0,NaN
2858,1814,2020-03-29,West Virginia,54,124,1,11,1,Wood County,-40.0,-12.0,-21.0,NaN,-31.0,9.0


In [40]:
all_oregon = df_all.loc[df_all['state'] == "Oregon"]

In [41]:
all_oregon

,index_row,date,state,fips,cases,deaths,new_cases,new_deaths,country,retail,grocery_and_pharmacy,parks,transit_stations,workplaces,residential
438,553,2020-02-28,Oregon,41,1,0,1,0,United States,8.0,4.0,24.0,7.0,3.0,-1.0
439,554,2020-02-29,Oregon,41,1,0,0,0,United States,7.0,5.0,8.0,6.0,3.0,-1.0
440,555,2020-03-01,Oregon,41,2,0,1,0,United States,17.0,9.0,23.0,9.0,3.0,-1.0
441,556,2020-03-02,Oregon,41,2,0,0,0,United States,8.0,7.0,14.0,2.0,3.0,0.0
442,557,2020-03-03,Oregon,41,2,0,0,0,United States,10.0,12.0,40.0,2.0,1.0,-1.0
443,558,2020-03-04,Oregon,41,2,0,0,0,United States,10.0,9.0,57.0,6.0,1.0,-1.0
444,559,2020-03-05,Oregon,41,2,0,0,0,United States,7.0,8.0,38.0,4.0,1.0,-1.0
445,560,2020-03-06,Oregon,41,2,0,0,0,United States,1.0,2.0,-3.0,-1.0,1.0,1.0
446,561,2020-03-07,Oregon,41,6,0,4,0,United States,5.0,3.0,16.0,4.0,3.0,0.0
447,562,2020-03-08,Oregon,41,13,0,7,0,United States,10.0,5.0,24.0,5.0,1.0,0.0
